In [7]:
import datetime
import pandas
import urllib
import json
import time
import sys
import datetime
import time
import requests
from yahoo_earnings_calendar import YahooEarningsCalendar
import numpy as np
import os

In [10]:
today = datetime.date.today()

path = os.getcwd()

today = datetime.date.today()

path = os.getcwd()

print(path)

#get data from bloomberg
def get_data(url):
	req = urllib.request.Request(url)
	response = urllib.request.urlopen(req)
	file = response.read()
	result = json.loads(file)
	result = pandas.DataFrame([result])
	return result

sandbox = "https://sandbox.iexapis.com"

sb_api_key = "Tpk_db75edaa9a9b4b2891dff83893eff6d6"

production = "https://cloud.iexapis.com/"

prod_api_key = "pk_1de77928d18b46e1b733bfa018321964"

base = datetime.datetime.today()

date_calendar = [base + datetime.timedelta(days=x) for x in range(7)]

date_from = date_calendar[0].date()

date_to = date_calendar[-1].date()

yec = YahooEarningsCalendar()

earnings = yec.earnings_between(date_from, date_to)

# earnings = json.loads(earnings)

eps_df = pandas.DataFrame.from_records(earnings)

/Users/wfclark/Azimuth/Trader/Collector


In [11]:
ticker_ary = []
epsEstimate = []
marketCap = []
peRatio = []
price = []
debt = []
priceToSales = []
priceToBook = []
sharesOutstanding = []
day5ChangePercent = []

In [12]:
eps_df = eps_df[['companyshortname', 'ticker', 'epsestimate', 'startdatetime', 'startdatetimetype']]

In [13]:
print(len(eps_df))

217


In [14]:
print(eps_df.head(20))

                      companyshortname ticker  epsestimate  \
0           Longevity Acquisition Corp  LOACU          NaN   
1                          Accuray Inc   ARAY        -0.07   
2              Zions Bancorporation NA   ZION          NaN   
3       First Community Bankshares Inc   FCBC         0.52   
4         United Financial Bancorp Inc   UBNK          NaN   
5        Southern First Bancshares Inc   SFST         0.85   
6      Meridian Bancorp Inc (MARYLAND)   EBSB         0.33   
7                TrustCo Bank Corp N Y   TRST         0.15   
8                   BankFinancial Corp   BFIN         0.21   
9               Timberland Bancorp Inc   TSBK          NaN   
10     Western New England Bancorp Inc   WNEB         0.13   
11           Bank of Commerce Holdings   BOCH         0.25   
12                 Fifth Third Bancorp  FITBI          NaN   
13                      BancFirst Corp   BANF         1.04   
14             Old Line Bancshares Inc   OLBK         0.54   
15      

In [15]:

for i in range(len(eps_df)):


	ticker = eps_df['ticker'].loc[i]
	price_url = "https://sandbox.iexapis.com/v1/stock/{}/quote/?token=Tpk_db75edaa9a9b4b2891dff83893eff6d6".format(str(eps_df['ticker'].loc[i]).lower())
	price_result = get_data(price_url)
	stats_url = "https://sandbox.iexapis.com/v1/stock/{}/advanced-stats/?token=Tpk_db75edaa9a9b4b2891dff83893eff6d6".format(str(eps_df['ticker'].loc[i]).lower())
	stats_result = get_data(stats_url)

	try:
		priceToSales.append(stats_result["priceToSales"].values[0])
		priceToBook.append(stats_result["priceToBook"].values[0])
		sharesOutstanding.append(stats_result["sharesOutstanding"].values[0])
		day5ChangePercent.append(stats_result["day5ChangePercent"].values[0])
		debt.append(stats_result["currentDebt"].values[0])
		epsEstimate.append(eps_df['epsestimate'].loc[i])
		marketCap.append(price_result["marketCap"].values[0])
		peRatio.append(price_result["peRatio"].values[0])
		price.append(price_result["latestPrice"].values[0])
		ticker_ary.append(ticker)
	except Exception as e:
		print(e)
		pass



HTTPError: HTTP Error 404: Not Found

In [16]:

print(len(ticker_ary), len(marketCap), len(peRatio), len(price), len(debt), len(priceToSales), len(priceToBook), len(sharesOutstanding), len(day5ChangePercent))

197 197 197 197 197 197 197 197 197


In [17]:
fin_analysis = pandas.DataFrame({"ticker": ticker_ary, "epsEstimate": epsEstimate, "marketCap": marketCap, 
                                 "peRatio": peRatio, "price": price, "debt": debt, "priceToSales": priceToSales , 
                                 "priceToBook" : priceToBook, "sharesOutstanding" : sharesOutstanding, "day5ChangePercent" : day5ChangePercent})
fin_analysis["epsEstimate"] = fin_analysis["epsEstimate"].fillna(value = -1)
fin_analysis["price"] = fin_analysis["price"].fillna(value = -1)
fin_analysis["peRatio"] = fin_analysis["peRatio"].fillna(value = -1)
fin_analysis["priceToBook"] = fin_analysis["priceToBook"].fillna(value = -1)
fin_analysis["priceToSales"] = fin_analysis["priceToSales"].fillna(value = -1)
fin_analysis["debt"] = fin_analysis["debt"].fillna(value = -1)
fin_analysis["day5ChangePercent"] = fin_analysis["day5ChangePercent"].fillna(value = -1)


In [22]:
import boto3
from io import StringIO

In [23]:
s3 = boto3.client("s3")

In [25]:
bucket = 'azimuthcollector'
csv_buffer = StringIO()
fin_analysis.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, f'azimuth_trades_{today}.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'B47BD14A930636CA',
  'HostId': 'SCm7shLhIru/nj8n7cIKMRVmWGfQyWMz39Jk5ZTGpIOsQFgq5YZy9MXB6CMVSMv22WZ+eQiIpc4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'SCm7shLhIru/nj8n7cIKMRVmWGfQyWMz39Jk5ZTGpIOsQFgq5YZy9MXB6CMVSMv22WZ+eQiIpc4=',
   'x-amz-request-id': 'B47BD14A930636CA',
   'date': 'Mon, 20 Jan 2020 18:56:09 GMT',
   'x-amz-version-id': 'SUjfnDHRp9pIh.haJTMdrEVmUVunzlKU',
   'etag': '"0e99dc3dd5dd424312d45e70311c99b2"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"0e99dc3dd5dd424312d45e70311c99b2"',
 'VersionId': 'SUjfnDHRp9pIh.haJTMdrEVmUVunzlKU'}